재귀를 까먹었나 왜 안넘어가지..

In [1]:
# breast cancer
from sklearn.datasets import load_breast_cancer
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

d = load_breast_cancer()

data = pd.DataFrame(d.data, columns=d.feature_names)
target = pd.DataFrame(d.target)

train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [2]:
d = data[data.columns[:10]]

In [3]:
d['mean area'].sort_values().index

Int64Index([101, 539, 538, 568,  46, 151, 314, 525,  61,  59,
            ...
            202, 164, 339, 122, 521,  82, 352, 180, 212, 461],
           dtype='int64', length=569)

In [4]:
def gini(a):
    c, n = np.unique(a, return_counts=True)
    tot = np.sum(n)
    res = 1
    for i in n:
        res *= i/tot
        
    return res

In [5]:
gini(target)

0.23376503037734625

In [6]:
d.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension'],
      dtype='object')

In [7]:
sorted(d['mean area'].unique())

[143.5,
 170.4,
 178.8,
 181.0,
 201.9,
 203.9,
 221.2,
 221.3,
 221.8,
 224.5,
 227.2,
 230.9,
 234.3,
 241.0,
 244.0,
 244.5,
 245.2,
 246.3,
 248.7,
 250.5,
 257.8,
 260.9,
 264.0,
 268.8,
 269.4,
 271.2,
 271.3,
 272.5,
 273.9,
 278.6,
 279.6,
 280.5,
 285.7,
 286.3,
 288.1,
 288.5,
 289.1,
 289.7,
 289.9,
 290.2,
 290.9,
 293.2,
 294.5,
 295.4,
 298.3,
 300.2,
 302.4,
 307.3,
 310.8,
 311.7,
 311.9,
 313.1,
 317.5,
 320.8,
 321.2,
 321.4,
 321.6,
 324.2,
 324.9,
 329.6,
 333.6,
 334.2,
 334.3,
 336.1,
 337.7,
 338.3,
 340.9,
 344.9,
 346.4,
 347.0,
 349.6,
 355.3,
 357.6,
 358.9,
 359.9,
 360.5,
 361.6,
 363.7,
 365.6,
 366.5,
 366.8,
 370.0,
 371.1,
 371.5,
 372.7,
 373.2,
 373.9,
 378.2,
 378.4,
 380.3,
 381.1,
 381.9,
 384.6,
 384.8,
 386.0,
 386.1,
 386.3,
 386.8,
 387.3,
 388.0,
 388.1,
 389.4,
 390.0,
 391.2,
 392.0,
 394.1,
 395.7,
 396.0,
 396.5,
 396.6,
 398.0,
 399.8,
 401.5,
 402.0,
 402.7,
 402.9,
 403.1,
 403.3,
 403.5,
 404.9,
 406.3,
 406.4,
 407.4,
 408.2,
 408.8,


In [8]:
d['mean area']

0      1001.0
1      1326.0
2      1203.0
3       386.1
4      1297.0
        ...  
564    1479.0
565    1261.0
566     858.1
567    1265.0
568     181.0
Name: mean area, Length: 569, dtype: float64

In [9]:
def making_tree(d):
    score = []
    index = []
    for c in d.columns:
        indices = d.sort_values(c).index
        temp_g = 1
        l = pd.Series([-1])
        r = pd.Series([-1])
        for i in range(len(d)):
            g = ((i+1)/len(indices)) * gini(target[0][indices[0:i]]) + (1 -(i+1)/len(indices)) * gini(target[0][indices[i:len(indices)]])
            if g < temp_g:
                temp_g = g
                l = indices[0:i]
                r = indices[i:len(indices)]
        score.append(temp_g)
        index.append((l.sort_values(),r.sort_values(), temp_g))
        
    return (d.columns[np.argmin(score)], index[np.argmin(score)])
        

In [10]:
z = making_tree(d)

In [11]:
z[1][2]

0.0759809841757674

In [16]:
g = gini(target)
hist = []
def forest(d):
    global hist
    print('goal')
    z = making_tree(d)
    hist.append(z)
    
    left = making_tree(d.loc[z[1][0],:])
    right = making_tree(d.loc[z[1][1], :])
    
    print(z[1][2])
    print(left[1][2])
    print(right[1][2])
    
    if left[1][2] < z[1][2]:
        forest(d.loc[z[1][0],:])
        return 
    if right[1][2] < z[1][2]:
        forest(d.loc[z[1][1],:])
    else:
        return

In [17]:
forest(d)

goal
0.0759809841757674
0.0367388581203663
0.07833124623547336
goal
0.0367388581203663
0.023410138924093743
0.10033545098480162
goal
0.023410138924093743
0.015139990582208503
0.12198347107438016
goal
0.015139990582208503
0.007334283193073684
0.04768269610905957
goal
0.007334283193073684
0.004219333524849181
0.03993055555555555
goal
0.004219333524849181
0.004237211407876867
1


In [14]:
# 객체 만들기
## fit >> data 입력시 tree 분기
## 가지치기 미시행먼저 하기


In [15]:
class sample:
    self.left = None
    self.right = None
    self.gini = None
    self.core = None
    def __init__(self, data):
        self.data = data
        res = making_tree(data)
        self.gini = res[1][2]
        self.core = res[0]
        self.left = res[1][0]
        self.right = res[1][1]
    
    def making_tree(d):
        score = []
        index = []
        for c in d.columns:
            indices = d.sort_values(c).index
            temp_g = 1
            for i in range(len(d)):
                g = ((i+1)/len(indices)) * gini(target[0][indices[0:i]]) + (1 -(i+1)/len(indices)) * gini(target[0][indices[i:len(indices)]])
                if g < temp_g:
                    temp_g = g
                    l = indices[0:i]
                    r = indices[i:len(indices)]

            score.append(temp_g)
            index.append((l.sort_values(),r.sort_values(), temp_g))

        return (d.columns[np.argmin(score)], index[np.argmin(score)])

    
    def fit(self, max_depth=9, gamma = 0.01):
        left_tree = Tree(self.data.iloc[self.left,:])
        if self.gini - left_tree.gini > gamma:
            self.left = left_tree.fit()
        
        

NameError: name 'self' is not defined

In [ ]:
k = sample('a', 'b')

In [ ]:
k.name